# language tools

Converting between languages, parsing languages computer languages, demonstrating tools like syntax hilighters, and outputting the diffed code


TODO: use syntax tree to convert from and to any language


use a common library for language generation 

https://bitbucket.org/birkenfeld/pygments-main/src/default/pygments/lexers/haskell.py

https://github.com/PrismJS/prism/blob/master/components/prism-markdown.js

http://qbnz.com/highlighter/

https://stackoverflow.com/questions/171924/how-do-i-syntax-check-a-bash-script-without-running-it

bash -n scriptname

Improve on the babel-preset-php pattern of replacing code with a tree, just use templates and the tree creator on regular code blocks

https://gitlab.com/kornelski/babel-preset-php/blob/master/lib/trans.js

```
function translateAssignBlock(node, scope) {
    if (isVariableCreatingAssignment(node, scope)) {
        return makeVariableDeclaration(node.left.name, "var", translateExpr(node.right, scope), node.loc, scope);
    }
    return wrapInStatement(translateAssign(node, scope));
}
```

should be something like:
```
var ASSIGMENT_EXPRESSION = (leftName, right) => {
    var leftName = right;
}

function translateAssignBlock(node, scope) {
    if(//Assignment[/Identifier].includes(node)) {
        return getAST(ASSIGMENT_EXPRESSION, node)
    }
    return getAST(STATEMENT, node)
}
```

that way when they are written in multiple languages we can transpile any language, maybe

use this library as a reference for lots of languages https://github.com/seaneshbaugh/rosetta-euler

automatically translate https://projecteuler.net/archives to any language


TODO: use babel and a transpiling kernel from babel-preset-php, 

TODO: make a generic translator that does manipulations on queries based on templates






In [ ]:
// sample definition for MatLab



In [ ]:
// definition for nodejs



## TODO: babel transpiler plugin



## syntax tree tools?




### justification

Just so it can be parsed with XPath instead. Get them all working the same!




Crap modules I won't need any more:
- css
- ASTQ
- Acorn
- esprima
- escodegen
- esquery
- jsonpath
- jsonquery
- jspath
- json:select
- jsel

TODO: get rid of jsonpath-plus, jsonstream, jspath, 



```
/* expected output `
[ Script {
    type: 'Program',
    body: [ [FunctionDeclaration], [ExpressionStatement] ],
    sourceType: 'script',
    range: [ 1, 45 ],
    comments: [],
    tokens:

<program type="program">
    <function>
    </function>
    <expression>
    </expression>
</program>

*/
```



### convert a syntax tree to html?

Reasoning. Every atom is made up of smaller component parts. This conversion creates a tree that breaks down to &lt;elements&gt and attr="ibutes", just like neutrons and protons.


In [113]:

function statementToElement(statement) {
    var {type} = statement;
    var result = ``, attrs = ``;
    var isList = true;
    var otherKeys = [];
    for(var i in statement) {
        if(parseInt(i) + '' === i + '') {
            result += statementToElement(statement[i])
            continue;
        }
        isList = false;
        // if the property is an Object, print out as a child
        // TODO: replace this type property when converting other trees
        //   should be like getEl() statementToElement(() => el.type)
        if(typeof statement[i].type !== 'undefined') {
            result += `<${i}-object>${statementToElement(statement[i])}</${i}-object>`
            otherKeys.push(i);
        } else if (typeof statement[i][0] !== 'undefined'
        // or if the property is an Array, print them out as child elements
                   && typeof statement[i][0].type !== 'undefined') {
            result += `<${i}-list>${statementToElement(statement[i])}</${i}-list>`
            otherKeys.push(i);
        } else {
            // if the property is not an object, i.e. native types
            attrs +=  `
${i}="${statement[i]}"`
        }
    }
    // print out the other keys just for fun matching
    if(!isList) attrs += otherKeys.map(k => k + '=""').join('');
    
    return isList ? result : `
<${type}${attrs}>${result}</${type}>`
}

// expects a string or a tree from JSDOM
function elementToStatement(body) {
    if(typeof body === 'string') {
        var dom = new JSDOM(body);
        body = dom.window.document.body;
    }
    if(body.nodeName === '#text') {
        return body.nodeValue;
    }
    var attrs = body.getAttributeNames();
    if(attrs.length === 0) {
        return Array.from(body.childNodes).map(c => elementToStatement(c))
    }
    var result = attrs.reduce((obj, p) => {
        var propToNodes = Array.from(body.childNodes)
           .filter(n => {
               return n.nodeName.toLowerCase() === p.toLowerCase() + '-object'
                   || n.nodeName.toLowerCase() === p.toLowerCase() + '-list'
           });
        if(propToNodes.length === 1
           && propToNodes[0].nodeName.substr(-7) === '-object') {
            obj[p] = elementToStatement(propToNodes[0].childNodes[0]);
        } else if(propToNodes.length > 0) {
            // converted from an array of childNodes above
            obj[p] = elementToStatement(propToNodes[0]);
        } else {
            obj[p] = body.getAttribute(p);
        }
        return obj;
    }, {})
    return result;
}

module.exports = {
    statementToElement,
    elementToStatement,
};


{ statementToElement: [Function: statementToElement],
  elementToStatement: [Function: elementToStatement] }

#### test tree to html?

In [114]:
var importer = require('../Core');

// TODO: make this dependency obsolete
var esquery = require('esquery');
var esprima = require('esprima'); 
var ASTQ  = require("astq");
var astq = new ASTQ();

function testASTArray(search) {
    //var code = importer.interpret(search).code;
    var code = `
function name(params) {
    
}
console.log()
`
    var ctx = esprima.parse(code, {range: true, tokens: true,
                        comment: true, whitespace: true})
    
    // TODO: demonstrate
    console.log(astq.query(ctx, '*')[0].body);
    console.log(esquery(ctx, '*').length);
    
    var page = statementToElement(ctx);
    console.log(elementToStatement(page)[0].body);
    return page;    
}

module.exports = testASTArray;

function htmlEntities(str) {
    return String(str)
        .replace(/&/g, '&amp;')
        .replace(/</g, '&lt;')
        .replace(/>/g, '&gt;')
        .replace(/"/g, '&quot;');
}

if(typeof $$ !== 'undefined') {
    var html = testASTArray('*', 'convert a tree to html')
    $$.html('<pre>' + htmlEntities(html) + '</pre>')
}


[ FunctionDeclaration {
    type: 'FunctionDeclaration',
    id:
     Identifier { type: 'Identifier', name: 'name', range: [Array] },
    params: [ [Identifier] ],
    body:
     BlockStatement { type: 'BlockStatement', body: [], range: [Array] },
    generator: false,
    expression: false,
    async: false,
    range: [ 1, 31 ] },
  ExpressionStatement {
    type: 'ExpressionStatement',
    expression:
     CallExpression {
       type: 'CallExpression',
       callee: [StaticMemberExpression],
       arguments: [],
       range: [Array] },
    range: [ 32, 45 ] } ]
10
[ '\n',
  { type: 'FunctionDeclaration',
    generator: 'false',
    expression: 'false',
    async: 'false',
    range: '1,31',
    id: [ '\n', [Object] ],
    params: [ '\n', [Object] ],
    body: [ '\n', [Object] ] },
  '\n',
  { type: 'ExpressionStatement',
    range: '32,45',
    expression: [ '\n', [Object] ] } ]


<Program
type="Program"
sourceType="script"
range="1,45"
comments=""body=""tokens=""><body-list>
<FunctionDeclaration
type="FunctionDeclaration"
generator="false"
expression="false"
async="false"
range="1,31"id=""params=""body=""><id-object>
<Identifier
type="Identifier"
name="name"
range="10,14"></Identifier></id-object><params-list>
<Identifier
type="Identifier"
name="params"
range="15,21"></Identifier></params-list><body-object>
<BlockStatement
type="BlockStatement"
body=""
range="23,31"></BlockStatement></body-object></FunctionDeclaration>
<ExpressionStatement
type="ExpressionStatement"
range="32,45"expression=""><expression-object>
<CallExpression
type="CallExpression"
arguments=""
range="32,45"callee=""><callee-object>
<MemberExpression
type="MemberExpression"
computed="false"
range="32,43"object=""property=""><object-object>
<Identifier
type="Identifier"
name="console"
range="32,39"></Identifier></object-object><property-object>
<Identifier
type="Identifier"
name="log"
range="40,43"></Identifier></property-object></MemberExpression></callee-object></CallExpression></expression-object></ExpressionStatement></body-list><tokens-list>
<Keyword
type="Keyword"
value="function"
range="1,9"></Keyword>
<Identifier
type="Identifier"
value="name"
range="10,14"></Identifier>
<Punctuator
type="Punctuator"
value="("
range="14,15"></Punctuator>
<Identifier
type="Identifier"
value="params"
range="15,21"></Identifier>
<Punctuator
type="Punctuator"
value=")"
range="21,22"></Punctuator>
<Punctuator
type="Punctuator"
value="{"
range="23,24"></Punctuator>
<Punctuator
type="Punctuator"
value="}"
range="30,31"></Punctuator>
<Identifier
type="Identifier"
value="console"
range="32,39"></Identifier>
<Punctuator
type="Punctuator"
value="."
range="39,40"></Punctuator>
<Identifier
type="Identifier"
value="log"
range="40,43"></Identifier>
<Punctuator
type="Punctuator"
value="("
range="43,44"></Punctuator>
<Punctuator
type="Punctuator"
value=")"
range="44,45"></Punctuator></tokens-list></Program>

#### TODO test tree to html with astq

TODO: convert between astq and other tree libraries

replacing that functionality I noticed in babel somewhere



In [106]:
// reserved words

var keywords = ['type', 'id', 'params',
                'body', 'generator', 'async',
                'range', 'expression', 'callee',
                'arguments'];



expected output


```
FunctionDeclaration {
    type: 'FunctionDeclaration',
    id:
     Identifier { type: 'Identifier', name: 'name', range: [Array] },
    params: [ [Identifier] ],
    body:
     BlockStatement { type: 'BlockStatement', body: [], range: [Array] },
    generator: false,
    expression: false,
    async: false,
    range: [ 1, 31 ] },
  ExpressionStatement {
    type: 'ExpressionStatement',
    expression:
     CallExpression {
       type: 'CallExpression',
       callee: [StaticMemberExpression],
       arguments: [],
       range: [Array] },
    range: [ 32, 45 ] } ]
```


### generate a synax tree from documentation


#### test language translator

TODO: Creating an interface like this becomes a simple http request to download and create a language compatible pattern, do in C# and in Node




In [ ]:

var wireKernelInterface = {
    ...metaKernelInterface,
    // implement all requests, these are required by the meta kernel
    execute_request, inspect_request, complete_request,
    history_request, is_complete_request, shutdown_request,
    // requests maybe not required right now?
    kernel_info_request, interrupt_request, input_request,
    connect_request, comm_info_request, kernel_info_request,

    // implement all replys
    execute_reply, inspect_reply, complete_reply,
    history_reply, kernel_info_reply, is_complete_reply,
    connect_reply, comm_info_reply, kernel_info_reply,
    shutdown_reply, interrupt_reply, input_reply,

    // a few extra protocol methods
    display_data, update_display_data, execute_input,
    execute_result, error, status, clear_output,
    comm_msg, comm_close
}

module.exports = wireKernelInterface;
    